In [1]:
from base64 import b64encode
from google.colab import files
from google.colab.patches import cv2_imshow
from IPython.display import HTML
from PIL import Image
from tqdm.notebook import tqdm
import cv2
import numpy as np
import os
import torch
import matplotlib.pyplot as plt

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# It may take a while...
!wget "https://documents.epfl.ch/groups/c/cv/cvlab-pom-video1/www/campus4-c0.avi"

--2021-07-02 05:32:25--  https://documents.epfl.ch/groups/c/cv/cvlab-pom-video1/www/campus4-c0.avi
Resolving documents.epfl.ch (documents.epfl.ch)... 128.178.222.197, 2001:620:618:1de:1:80b2:dec5:1
Connecting to documents.epfl.ch (documents.epfl.ch)|128.178.222.197|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 33448124 (32M) [application/octet-stream]
Saving to: ‘campus4-c0.avi’

campus4-c0.avi      100%[===================>]  31.90M  1012KB/s    in 39s     

2021-07-02 05:33:05 (832 KB/s) - ‘campus4-c0.avi’ saved [33448124/33448124]



In [3]:
model = torch.hub.load('ultralytics/yolov5', 'custom', path='/content/drive/MyDrive/YOLOv5/face-mask/best0708_2.pt',force_reload=True) #force_reload 不知道有何用

Downloading: "https://github.com/ultralytics/yolov5/archive/master.zip" to /root/.cache/torch/hub/master.zip


requirements: PyYAML>=5.3.1 not found and is required by YOLOv5, attempting auto-update...


YOLOv5 🚀 2021-7-8 torch 1.9.0+cu102 CUDA:0 (Tesla T4, 15109.75MB)



  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13

requirements: 1 package updated per /root/.cache/torch/hub/ultralytics_yolov5_master/requirements.txt
requirements: ⚠️ Restart runtime or rerun command for updates to take effect



Fusing layers... 
Model Summary: 484 layers, 88404072 parameters, 0 gradients
Adding AutoShape... 


In [ ]:
img = '/content/drive/MyDrive/YOLOv5/Person/2.jpg'
model.conf = 0.1
results = model(img)
results.print()

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


image 1/1: 855x1140 6 no_masks, 26 with_masks, 31 persons
Speed: 388.3ms pre-process, 160.2ms inference, 91.3ms NMS per image at shape (1, 3, 480, 640)


In [ ]:
results.save()

Saved 2.jpg to runs/hub/exp


In [ ]:
results.pandas().xyxy[0]

,xmin,ymin,xmax,ymax,confidence,class,name
0,847.875000,164.097656,1091.906250,810.468750,0.951660,2,person
1,749.906250,98.135742,891.515625,317.730469,0.937500,2,person
2,168.328125,216.978516,236.906250,290.566406,0.930176,1,with_mask
3,590.039062,281.214844,651.492188,345.339844,0.925293,1,with_mask
4,440.859375,159.199219,500.531250,220.875000,0.921387,1,with_mask
...,...,...,...,...,...,...,...
58,233.566406,111.161133,298.136719,221.097656,0.148682,0,no_mask
59,295.019531,49.875000,388.089844,199.722656,0.139526,0,no_mask
60,2.031738,43.417969,110.771484,804.234375,0.138184,2,person
61,539.718750,130.031250,616.312500,256.945312,0.134888,1,with_mask


In [70]:
model.names

['no_mask', 'mask', 'person']

In [71]:
# Load model
model = torch.hub.load('ultralytics/yolov5', 'yolov5x',
                       pretrained=True, verbose=False) #加載預訓練的 YOLOv5s 模型
                       #verbose Default is True. # 如果為 False，則忽略有關命中本地緩存的消息
#model.cuda('cuda:0'); #?? #大概只是顯示GPU 沒有必要執行 #真的沒意義

requirements: PyYAML>=5.3.1 not found and is required by YOLOv5, attempting auto-update...


YOLOv5 🚀 2021-7-8 torch 1.9.0+cu102 CUDA:0 (Tesla T4, 15109.75MB)




requirements: 1 package updated per /root/.cache/torch/hub/ultralytics_yolov5_master/requirements.txt
requirements: ⚠️ Restart runtime or rerun command for updates to take effect



Exception: ignored

In [4]:
def display_video(path): #有空再來研究
    '''Display video in Colab.'''
    compressed_path = path.split('.')[0] #split 切片
    compressed_path = 'compressed_' + compressed_path + '.mp4'

    if os.path.exists(compressed_path):
        os.remove(compressed_path)

    # Convert video
    os.system(f"ffmpeg -i {path} -vcodec libx264 {compressed_path}")

    # Show video
    mp4 = open(compressed_path,'rb').read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
    return HTML("""
    <video width=400 controls>
        <source src="%s" type="video/mp4">
    </video>
    """ % data_url)
    #print(HTML)

In [5]:
filename = '/content/drive/MyDrive/YOLOv5/Person/ppp.mp4'

In [6]:
def calculate_distance(point1, point2):
    '''Calculate usual distance.'''
    x1, y1 = point1
    x2, y2 = point2
    return np.linalg.norm([x1 - x2, y1 - y2])

In [7]:
def convert_to_bird(centers, M):
    '''Apply the perpective to the bird's-eye view.'''
    centers = [cv2.perspectiveTransform(np.float32([[center]]), M) for center in centers.copy()]
    centers = [list(center[0, 0]) for center in centers.copy()]
    return centers

In [8]:
def img_text(img,border_size,person,lineCount, maskCount, nomaskCount):
    ratio = nomaskCount/(maskCount+nomaskCount+0.000001) #避免分母為0
    ratio = round(ratio,3)*100
    ratio = round(ratio,2)
    #print(f"np isnan ? : {np.isnan(ratio)}")
    border_text_color=[255,255,255]
    #收集資料

    
    text="social distance status:  "
    cv2.putText(img ,text , (0, int(border_size-10)), cv2.FONT_HERSHEY_SIMPLEX,0.8,border_text_color, 2)  
    if lineCount >=3: #條件待改良 ，text擺放位置要再重新設計
        text = "Danger !"
        cv2.putText(img,text, (300, int(border_size-10)), cv2.FONT_HERSHEY_SIMPLEX,0.8,[26,13,247], 2)
              #cv2.putText(影像, 文字, 座標, 字型, 大小, 顏色, 線條寬度, 線條種類)
    elif 3> lineCount >=1:
        text = "Warning !"
        cv2.putText(img,text, (300, int(border_size-10)), cv2.FONT_HERSHEY_SIMPLEX,0.8,[0,255,255], 2)

    else:
        text = "Safe "
        cv2.putText(img,text, (300, int(border_size-10)), cv2.FONT_HERSHEY_SIMPLEX,0.8,[0,255,0], 2)
    
    #口罩配戴警告
    text="mask status         :  "
    cv2.putText(img ,text , (0, int(border_size-50)), cv2.FONT_HERSHEY_SIMPLEX,0.8,border_text_color, 2)
    if ratio>=0.1 and nomaskCount>=3: #社交距離 警告 ，條件待改良 ，text擺放位置要再重新設計
        text = "Danger !"
        cv2.putText(img,text, (300, int(border_size-50)), cv2.FONT_HERSHEY_SIMPLEX,0.8,[26,13,247], 2)
        warning = 'Danger'
        
    elif ratio!=0 and np.isnan(ratio)!=True: #ratio 不含空值
        text = "Warning !"
        cv2.putText(img,text, (300, int(border_size-50)), cv2.FONT_HERSHEY_SIMPLEX,0.8,[0,255,255], 2)
        warning = 'Warning'

    else:
        text = "Safe "
        cv2.putText(img,text, (300, int(border_size-50)), cv2.FONT_HERSHEY_SIMPLEX,0.8,[0,255,0], 2)
        warning = 'Safe'
    
    #口罩配戴比率  #尚未有比率計算
    #print(f"this is ratio: {ratio}")
    text="mask wearing rate   : {}%".format(ratio)
    cv2.putText(img ,text , (0, int(border_size-90)), cv2.FONT_HERSHEY_SIMPLEX,0.8,border_text_color, 2)    
    
    #多少有口罩
    text="MaskCount: {}".format(maskCount)
    cv2.putText(img ,text , (435, int(border_size-10)), cv2.FONT_HERSHEY_SIMPLEX,0.8,border_text_color, 2) 

    #多少沒配戴口罩
    text="NoMaskCount: {}".format(nomaskCount)
    cv2.putText(img ,text , (435, int(border_size-50)), cv2.FONT_HERSHEY_SIMPLEX,0.8,border_text_color, 2) 

    #多少違反社交距離的紅線 #也許之後不用 可以刪除
    # text="redLine: {} ".format(lineCount) 
    # cv2.putText(img ,text , (425, int(border_size-90)), cv2.FONT_HERSHEY_SIMPLEX,0.8,border_text_color, 2)    

    
    return img, ratio ,warning

In [9]:
def bird_detect_people_on_frame(img, confidence, distance, width, height,border_size, mask_rate_list, mask_warning_list,
                                region=None, dst=None):
    results = model([img[:, :, ::-1]])  # Pass the frame through the model and get the boxes
    
    xyxy = results.xyxy[0].cpu().numpy()  # xyxy are the box coordinates
    #          x1 (pixels)  y1 (pixels)  x2 (pixels)  y2 (pixels)   confidence        class
    # tensor([[7.47613e+02, 4.01168e+01, 1.14978e+03, 7.12016e+02, 8.71210e-01, 0.00000e+00],
    #         [1.17464e+02, 1.96875e+02, 1.00145e+03, 7.11802e+02, 8.08795e-01, 0.00000e+00],
    #         [4.23969e+02, 4.30401e+02, 5.16833e+02, 7.20000e+02, 7.77376e-01, 2.70000e+01],
    #         [9.81310e+02, 3.10712e+02, 1.03111e+03, 4.19273e+02, 2.86850e-01, 2.70000e+01]])

    xyxy = xyxy[xyxy[:, 4] >= confidence]  # Filter desired confidence
    nomask_xyxy = xyxy[xyxy[:, 5] == 0]
    nomask_xyxy = nomask_xyxy[:, :6]

    mask_xyxy = xyxy[xyxy[:, 5] == 1] #有配戴口罩 座標
    mask_xyxy = mask_xyxy[:, :6]
    
    xyxy = xyxy[xyxy[:, 5] == 2]  # Consider only people
    xyxy = xyxy[:, :6]
    #辨識到人的方框總數
    listclass = xyxy[:,-1].tolist()
    toint = [int(i) for i in listclass]
    person =toint.count(2)
    contactCount = 0
    maskCount = 0
    nomaskCount = 0

    # Calculate the centers of the circles #計算圓心
    # They will be the centers of the bottom of the boxes #它們將是盒子底部的中心
    centers = []
    for x1, y1, x2, y2, conf, label in xyxy:
        center = [np.mean([x1, x2]), int(y2)+border_size]
        #print(f"this is center: {center}")
        centers.append(center)
        # text = "{}: {:.4f}".format(model.names[int(label)],conf)  #顯示 辨識人的 name & conf
        # img = cv2.putText(img, text, (x1, int(y1)+border_size-5), cv2.FONT_HERSHEY_SIMPLEX,0.5, (0, 255, 0), 1)  #這邊顏色先隨便上
    # We create two transformations
    if region is None:
        # The region on the original image
        region = np.float32([[144, 130], [666, 129], [width, height], [0, height]])
    if dst is None:
        # The rectangle we want the image to be trasnformed to
        dst = np.float32([[0, 0], [width, 0], [width, 3*width], [0, 3*width]]) #整張圖片
    # The first transformation is straightforward: the region to the rectangle
    # as thin the example before #第一個轉換是直接將區域轉換為矩形，就像之前的例子一樣薄 
    M = cv2.getPerspectiveTransform(region, dst)

    # The second transformation is a trick, because, using the common transformation,
    # we can't draw circles at left of the region.
    # This way, we flip all things and draw the circle at right of the region,
    # because we can do it.
    region_flip = region*np.float32([-1, 1]) + np.float32([width, 0])
    dst_flip = dst*np.float32([-1, 1]) + np.float32([width, 0])
    M_flip = cv2.getPerspectiveTransform(region_flip, dst_flip)

    # Convert to bird
    # Now, the center of the circles will be positioned on the rectangle
    # and we can calculate the usual distance #圓的中心將位於矩形上，我們可以計算通常的距離 
    bird_centers = convert_to_bird(centers, M)

    # We verify if the circles colide
    # If so, they will be red
    colors = ['green']*len(bird_centers)
    for i in range(len(bird_centers)):
        for j in range(i+1, len(bird_centers)):
            dist = calculate_distance(bird_centers[i], bird_centers[j])
            if dist < distance:
                contactCount +=1
                colors[i] = 'red'
                colors[j] = 'red'

    # We draw the circles
    # Because we have two transformation, we will start with two empty
    # images ("overlay" images) to draw the circles
    overlay = np.zeros((3*width, 4*width, 3), np.uint8)
    overlay_flip = np.zeros((3*width, 4*width, 3), np.uint8)
    for i, bird_center in enumerate(bird_centers):
        if colors[i] == 'green':
            color = (0, 255, 0)
        else:
            color = (0, 0, 255)
        x, y = bird_center
        x = int(x)
        y = int(y)
        if x >= int(distance/2+15/2): #這甚麼神奇算式
            # If it's the case the circle is inside or at right of our region
            # we can use the normal overlay image
            overlay = cv2.circle(overlay, (x, y), int(distance/2),
                                  color, 15, lineType=cv2.LINE_AA)
        else:
            # If the circle is at left of the region,
            # we draw the circle inverted on the other overlay image
            x = width - x
            overlay_flip = cv2.circle(overlay_flip, (x, y), int(distance/2),
                                  color, 15, lineType=cv2.LINE_AA)

    # We apply the inverse transformation to the overlay
    overlay = cv2.warpPerspective(overlay, M, (width, height),
                                  cv2.INTER_NEAREST, cv2.WARP_INVERSE_MAP)
    # We apply the inverse of the other transformation to the other overlay
    overlay_flip = cv2.warpPerspective(overlay_flip, M_flip, (width, height),
                                       cv2.INTER_NEAREST, cv2.WARP_INVERSE_MAP)
    # Now we "unflip" what the second overlay
    overlay_flip = cv2.flip(overlay_flip, 1)

    # 增加邊框
    img = cv2.copyMakeBorder(img, border_size,0,0,0, cv2.BORDER_CONSTANT)

    #有配戴口罩 座標
    for i, (x1, y1, x2, y2, c1, label) in enumerate(mask_xyxy):
        # Draw the boxes   
        #print(f"this is i: {i}")
        #if mask_colors[i] == 'green':
        mask_colors = (0, 255, 0)#綠
        maskCount += 1
        #else:
        #    mask_colors = (0, 0, 255)          
        img = cv2.rectangle(img, (int(x1), int(y1)+border_size), (int(x2), int(y2)+border_size), mask_colors, 2) #之後再看有沒有聰明一點方式轉成int
        
        text = "{}: {:.4f}".format(model.names[int(label)],c1)
        img = cv2.putText(img, text, (int(x1), int(y1)+border_size-5), cv2.FONT_HERSHEY_SIMPLEX,0.8, mask_colors, 2)
    
    #無配戴口罩 座標
    for i, (x1, y1, x2, y2, c1, label) in enumerate(nomask_xyxy):
        # Draw the boxes   
        mask_colors = (0, 0, 255)#紅
        nomaskCount += 1
        img = cv2.rectangle(img, (int(x1), int(y1)+border_size), (int(x2), int(y2)+border_size), mask_colors, 2) #之後再看有沒有聰明一點方式轉成int
        
        text = "{}: {:.4f}".format(model.names[int(label)],c1)
        img = cv2.putText(img, text, (int(x1), int(y1)+border_size-5), cv2.FONT_HERSHEY_SIMPLEX,0.8, mask_colors, 2) 
            #cv2.putText(影像, 文字, 座標, 字型, 大小, 顏色, 線條寬度, 線條種類)
    # text增加
    img_text(img, border_size, person, contactCount, maskCount, nomaskCount)[0]
    
    mask_rate_list.append(img_text(img, border_size, person, contactCount, maskCount, nomaskCount)[1])
    mask_warning_list.append(img_text(img, border_size, person, contactCount, maskCount, nomaskCount)[2])
    # We add all images
    img = cv2.addWeighted(img, 1, overlay, 1, 0)
    img = cv2.addWeighted(img, 1, overlay_flip, 1, 0)
    

    return img, mask_rate_list, mask_warning_list

In [36]:
a = ['bang','bang','apple','ggwp','ggwp']
rep = [0 if x =='bang' else x for x in a]
rep = [1 if x =='ggwp' else x for x in rep]
rep = [2 if x =='apple' else x for x in rep]
# Danger= 2 ，warning= 1 , Safe= 0
rep

[0, 0, 2, 1, 1]

In [10]:
def mask_rate_line_plot(mask_rate_list, frameCount):
  #折線圖
  fig = plt.figure()
  frame = range(1,frameCount+1)
  plt.plot(frame, mask_rate_list, color = 'blue')
  plt.xlabel('frame')
  plt.ylabel('rate')
  plt.ylim(0, 101)
  plt.title('mask rate')
  fig.set_figheight(3)
  fig.set_figwidth(10)
  plt.savefig('mask_rate_line_plot.png')

In [11]:
def mask_rate_scatter_plot(mask_rate_list, frameCount):
  #散點圖
  fig = plt.figure()     
  frame = range(1,frameCount+1)
  plt.scatter(frame, mask_rate_list, c="red")
  plt.xlabel("frame")
  plt.ylabel("rate")
  plt.ylim(0, 101)
  plt.title("mask rate")
  fig.set_figheight(4)
  fig.set_figwidth(8)
  plt.savefig('mask_rate_scatter_plot.png')

In [12]:
def mask_warning_line_plot(mask_warning_list_int, frameCount):
  #折線圖
  my_yticks = ['Safe','Warning','Danger']
  fig = plt.figure()
  frame = range(1,frameCount+1)
  plt.plot(frame, mask_warning_list_int, color = 'blue')
  plt.xlabel('frame')
  plt.ylabel('warning')
  #plt.ylim(0, 2)
  plt.yticks([0,1,2], my_yticks)
  plt.margins(0.08)
  plt.subplots_adjust(bottom=0.15)
  plt.title('mask rate')
  fig.set_figheight(3)
  fig.set_figwidth(10)
  plt.savefig('mask_warning_line_plot.png')

In [13]:
def bird_detect_people_on_video(filename, confidence=0.9, distance=160):
    # Capture video
    cap = cv2.VideoCapture(filename)

    border_size=125

    mask_rate_list = []
    mask_warning_list = []
    frameCount = 0
    # Get video properties
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)) +border_size

    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    if os.path.exists('bird_output.avi'):
        os.remove('bird_output.avi')
    out = cv2.VideoWriter('bird_output.avi', fourcc, fps, (width, height))

    # Iterate through frames
    vidlen = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    with tqdm(total=vidlen) as pbar:
        while cap.isOpened():
            # Read frame
            ret, frame = cap.read()
            if ret == True:
                # Detect people as a bird
                frame = bird_detect_people_on_frame(frame, confidence, distance,
                                                    width, height, border_size, mask_rate_list, mask_warning_list)[0]
                #print(f"mask_rate_list len :{len(mask_rate_list)}")
                frameCount += 1
                # Write frame to new video
                out.write(frame)
                pbar.update(1)
            else:
                break

    # print(f"count: {frameCount}")

    # Danger= 2 ，warning= 1 , Safe= 0
    mask_warning_list_int = [0 if x =='Safe' else x for x in mask_warning_list]
    mask_warning_list_int = [1 if x =='Warning' else x for x in mask_warning_list_int]
    mask_warning_list_int = [2 if x =='Danger' else x for x in mask_warning_list_int]
    # print(f"mask_warning_list_int len :{len(mask_warning_list_int)}")
    # print(f"mask_warning_list_int: {(mask_warning_list_int)}")
    # line plot
    mask_warning_line_plot(mask_warning_list_int, frameCount)
    # line plot
    mask_rate_line_plot(mask_rate_list, frameCount)
    # scatter plot
    mask_rate_scatter_plot(mask_rate_list, frameCount)
    # Release everything if job is finished
    cap.release()
    out.release()
    cv2.destroyAllWindows()

    #return mask_rate_list

In [14]:
bird_detect_people_on_video(filename, confidence=0.5, distance=120)

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


In [15]:
display_video('bird_output.avi')

Output hidden; open in https://colab.research.google.com to view.